# Import Libraries

In [ ]:
from llama_index.core import SimpleDirectoryReader,Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding
from pypdf import PdfReader
import numpy as np
from IPython.display import Markdown, display

# Load Data

We will be using custom pdf

In [ ]:
reader = PdfReader('path to a pdf file')

### In this work we will not be using our own custom nodes.

``` We will check for reserch paper pdf with keyword abstract and references. If present we will take content starting from abstract till beginning of references, otherwise we will consider the full pdf. The steps for nodes creattion is as follows: ```

- merge all sentences into a singe string
- convert to lower case character
- split into sentence 
- create nodes of 20 sentences with a overlap of 1 sentence

In [ ]:
documents_1 = ''
first_section = "abstract"
ignore_after = "references"
pg_no = 0
for page in reader.pages:
    pg_no += 1
    documents_1 += page.extract_text(0)
cleaned_string = documents_1.replace('\n', ' ')
cleaned_string = cleaned_string.lower()

start_index = cleaned_string.find(first_section)
end_index = cleaned_string.rindex(ignore_after)
cleaned_string = cleaned_string[start_index:end_index]

In [ ]:
sentence_list = cleaned_string.split('. ')
context_list = []
page_number_list = []
group_size = 20
overlap = 1
i = 0 
while True:
    group = sentence_list[i:i+group_size]
    text = '. '.join(group)
    if len(text)>10:
        context_list.append(text)
    i+=group_size-overlap
    if i>=len(sentence_list):
        break

In [ ]:
from llama_index.core import Document

documents = [Document(text=t) for t in context_list]
documents_for_summarization = [Document(text=t) for t in context_list[:3]]

# Specify LLM and Embedding Model

In [ ]:
Settings.llm = Ollama(model='phi3',request_timeout=3600.0,temperature=1)
Settings.embed_model = OllamaEmbedding(model_name='nomic-embed-text')

# Create Index

In [ ]:
from llama_index.core import SummaryIndex,VectorStoreIndex

summary_index = SummaryIndex(documents)
vector_index = VectorStoreIndex(documents_for_summarization)

# Create Query Engine

In [ ]:
vector_query_engine = vector_index.as_query_engine(streaming=True)
summary_query_engine = summary_index.as_query_engine(response_mode="tree_summarize",streaming=True)

In [ ]:
# define prompt viewing function
def display_prompt_dict(prompts_dict):
    for k, p in prompts_dict.items():
        text_md = f"**Prompt Key**: {k}<br>" f"**Text:** <br>"
        display(Markdown(text_md))
        print(p.get_template())
        display(Markdown("<br><br>"))

In [ ]:
prompts_dict = vector_query_engine.get_prompts()
display_prompt_dict(prompts_dict)

In [ ]:
prompts_dict = summary_query_engine.get_prompts()
display_prompt_dict(prompts_dict)

## Updating Prompt for Phi3

### Phi3 from `Microsoft` has a speicific prompting pattern as defined in the [Model Card](https://huggingface.co/microsoft/Phi-3-mini-4k-instruct) 

In [ ]:
from llama_index.core import PromptTemplate

In [ ]:
qa_prompt_tmpl_str = ("<|user|>\n"
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information and not prior knowledge, answer the query\n"
    "Query: {query_str}"
    " <|end|>\n"
    "<|assistant|>"
)
qa_prompt_tmpl = PromptTemplate(qa_prompt_tmpl_str)

vector_query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_prompt_tmpl}
)

In [ ]:
refine_prompt_tmpl_str = ("<|user|>\n"
    "The original query is as follows: {query_str}\n"
    "We have provided an existing answer: {existing_answer}\n"
    "We have the opportunity to refine the existing answer (only if needed) with some more context below.\n"
    "---------------------\n"
    "{context_msg}\n"
    "---------------------\n"
    "Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer."
    " <|end|>\n"
    "<|assistant|>"
)
refine_prompt_tmpl = PromptTemplate(refine_prompt_tmpl_str)

vector_query_engine.update_prompts(
    {"response_synthesizer:refine_template": refine_prompt_tmpl}
)

In [ ]:
summary_prompt_tmpl_str = ("<|user|>\n"
    "Context information from multiple sources is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the information from multiple sources and not prior knowledge, answer the query.\n"
    "Query: {query_str}"
    " <|end|>\n"
    "<|assistant|>"
)
summary_prompt_tmpl = PromptTemplate(summary_prompt_tmpl_str)

summary_query_engine.update_prompts(
    {"response_synthesizer:summary_template": summary_prompt_tmpl}
)

# Create Query Engine Tool

In [ ]:
from llama_index.core.vector_stores import MetadataFilters,MetadataFilter
from llama_index.core.tools import QueryEngineTool

In [ ]:
summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=('Useful for summarization questions')
)

query_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=('Useful for specific topic based questions')
)

In [ ]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector

agent = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,query_tool
    ],
    verbose=True

)

In [ ]:
response1 = agent.query("What is attention?")

In [ ]:
response1.print_response_stream()

In [ ]:
response2 = agent.query("Describe the paper to me")

In [ ]:
response2.print_response_stream()